In [15]:
import cv2
import numpy as np
import pandas as pd

# Function to extract features from a video
def extract_features_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    features = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize the frame to reduce complexity
        frame = cv2.resize(frame, (64, 64))  # Resize to 64x64 pixels
        # Flatten the frame and store it as feature vector
        feature = frame.flatten()
        features.append(feature)
    
    cap.release()
    return np.array(features)

# List of video paths
video_paths = [
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 1.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 2.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 3.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 4.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 5.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 6.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 7.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 8.mp4",
    "C:\\Users\\sivad\\OneDrive\\Desktop\\viedos\\video 9.mp4"
]

# Extract features from all videos and store them in a list
video_features = []
for video_path in video_paths:
    features = extract_features_from_video(video_path)
    video_features.append(np.mean(features, axis=0))  # Taking the average of features from the video

# Convert the list of features into a Pandas DataFrame
video_features_df = pd.DataFrame(video_features)

# Save the extracted features to a CSV file for later use
video_features_df.to_csv('video_features.csv', index=False)
print("Video features extracted and saved to 'video_features.csv'.")


Video features extracted and saved to 'video_features.csv'.


In [16]:
import numpy as np
import pandas as pd

# Load the extracted video features
df = pd.read_csv('video_features.csv')

# Generate random fraud labels (0 for non-fraud, 1 for fraud) for testing
# The number of labels should match the number of videos (9 in this case)
df['fraud'] = np.random.randint(0, 2, size=len(df))

# Save the labeled dataset to CSV for future use
df.to_csv('video_features_with_labels.csv', index=False)

# Now you can proceed with training models
print(df.head())


            0           1           2           3           4          5  \
0    9.558376    7.380711    6.807107   10.062606    7.487310   6.382403   
1   56.580161   51.666667   63.122261   64.928489   58.710496  70.284890   
2  109.174528  131.129717  143.155660   43.919811   50.495283  62.275943   
3  192.547101  131.172101  113.099638  189.728261  118.902174  96.554348   
4   68.221914   49.718447   56.184466   69.152566   50.603329  58.289875   

            6           7          8           9  ...       12279       12280  \
0   14.395939   10.592217   8.050761   11.148900  ...   35.549915   16.126904   
1   71.919262   63.198385  76.417532   79.448674  ...   82.681661   90.833910   
2   41.634434   49.884434  59.000000   51.195755  ...  162.129717  164.344340   
3  192.960145  118.927536  94.802536  194.472826  ...   63.423913  121.927536   
4   71.865465   52.467406  61.489598   74.266297  ...   88.703190   45.986130   

        12281       12282       12283       12284       

In [17]:
import pandas as pd
from IPython.display import FileLink

# Load your DataFrame (assuming you have already saved the file previously)
df = pd.read_csv('video_features_with_labels.csv')

# Optionally display the DataFrame to check its contents
print(df.head())

# Create a downloadable link for the CSV file
df.to_csv('video_features_with_labels.csv', index=False)

# Provide a link to download the CSV file
display(FileLink(r'video_features_with_labels.csv'))


            0           1           2           3           4          5  \
0    9.558376    7.380711    6.807107   10.062606    7.487310   6.382403   
1   56.580161   51.666667   63.122261   64.928489   58.710496  70.284890   
2  109.174528  131.129717  143.155660   43.919811   50.495283  62.275943   
3  192.547101  131.172101  113.099638  189.728261  118.902174  96.554348   
4   68.221914   49.718447   56.184466   69.152566   50.603329  58.289875   

            6           7          8           9  ...       12279       12280  \
0   14.395939   10.592217   8.050761   11.148900  ...   35.549915   16.126904   
1   71.919262   63.198385  76.417532   79.448674  ...   82.681661   90.833910   
2   41.634434   49.884434  59.000000   51.195755  ...  162.129717  164.344340   
3  192.960145  118.927536  94.802536  194.472826  ...   63.423913  121.927536   
4   71.865465   52.467406  61.489598   74.266297  ...   88.703190   45.986130   

        12281       12282       12283       12284       

C:\Users\sivad\video_features_with_labels.csv

In [18]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load the dataset with video features and labels
df = pd.read_csv('video_features_with_labels.csv')

# Check for class imbalance
print("Class Distribution Before Handling Imbalance:")
print(df['fraud'].value_counts())

# Prepare features (X) and target (y)
X = df.drop('fraud', axis=1)
y = df['fraud']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features to improve model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SMOTE with k_neighbors=1 to handle the small dataset
smote = SMOTE(k_neighbors=1)

# Apply SMOTE only to the training set
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Check class distribution after SMOTE
print("Class Distribution After SMOTE:")
print(pd.Series(y_train_smote).value_counts())

# Logistic Regression Model with class_weight set to 'balanced'
log_model = LogisticRegression(max_iter=1000, class_weight='balanced')
log_model.fit(X_train_smote, y_train_smote)

# Predict using the test set
y_pred_lr = log_model.predict(X_test_scaled)

# Calculate and print performance metrics
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, zero_division=1)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)

# Output the results
print(f'Logistic Regression Accuracy: {accuracy_lr}')
print(f'Logistic Regression Precision: {precision_lr}')
print(f'Logistic Regression Recall: {recall_lr}')
print(f'Logistic Regression F1-Score: {f1_lr}')



Class Distribution Before Handling Imbalance:
fraud
1    6
0    3
Name: count, dtype: int64
Class Distribution After SMOTE:
fraud
1    4
0    4
Name: count, dtype: int64
Logistic Regression Accuracy: 1.0
Logistic Regression Precision: 1.0
Logistic Regression Recall: 1.0
Logistic Regression F1-Score: 1.0


In [19]:
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('video_features_with_labels.csv')

# Prepare features (X) and target (y)
X = df.drop('fraud', axis=1)
y = df['fraud']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SMOTE with k_neighbors=1 to handle the small dataset
smote = SMOTE(k_neighbors=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# SVM Model
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_svm = svm_model.predict(X_test_scaled)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, zero_division=1)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

# Output the results for SVM
print(f'SVM Accuracy: {accuracy_svm}')
print(f'SVM Precision: {precision_svm}')
print(f'SVM Recall: {recall_svm}')
print(f'SVM F1-Score: {f1_svm}')



SVM Accuracy: 1.0
SVM Precision: 1.0
SVM Recall: 1.0
SVM F1-Score: 1.0


In [20]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree Model
tree_model = DecisionTreeClassifier(class_weight='balanced')
tree_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_tree = tree_model.predict(X_test_scaled)

accuracy_tree = accuracy_score(y_test, y_pred_tree)
precision_tree = precision_score(y_test, y_pred_tree, zero_division=1)
recall_tree = recall_score(y_test, y_pred_tree)
f1_tree = f1_score(y_test, y_pred_tree)

# Output the results for Decision Tree
print(f'Decision Tree Accuracy: {accuracy_tree}')
print(f'Decision Tree Precision: {precision_tree}')
print(f'Decision Tree Recall: {recall_tree}')
print(f'Decision Tree F1-Score: {f1_tree}')


Decision Tree Accuracy: 0.6666666666666666
Decision Tree Precision: 0.6666666666666666
Decision Tree Recall: 1.0
Decision Tree F1-Score: 0.8


In [21]:
from catboost import CatBoostClassifier

# CatBoost Model
cat_model = CatBoostClassifier(verbose=0)
cat_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_cat = cat_model.predict(X_test_scaled)

accuracy_cat = accuracy_score(y_test, y_pred_cat)
precision_cat = precision_score(y_test, y_pred_cat, zero_division=1)
recall_cat = recall_score(y_test, y_pred_cat)
f1_cat = f1_score(y_test, y_pred_cat)

# Output the results for CatBoost
print(f'CatBoost Accuracy: {accuracy_cat}')
print(f'CatBoost Precision: {precision_cat}')
print(f'CatBoost Recall: {recall_cat}')
print(f'CatBoost F1-Score: {f1_cat}')


CatBoost Accuracy: 0.6666666666666666
CatBoost Precision: 0.6666666666666666
CatBoost Recall: 1.0
CatBoost F1-Score: 0.8


In [22]:
from xgboost import XGBClassifier

# XGBoost Model
xgb_model = XGBClassifier()
xgb_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_xgb = xgb_model.predict(X_test_scaled)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, zero_division=1)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

# Output the results for XGBoost
print(f'XGBoost Accuracy: {accuracy_xgb}')
print(f'XGBoost Precision: {precision_xgb}')
print(f'XGBoost Recall: {recall_xgb}')
print(f'XGBoost F1-Score: {f1_xgb}')


XGBoost Accuracy: 0.6666666666666666
XGBoost Precision: 1.0
XGBoost Recall: 0.5
XGBoost F1-Score: 0.6666666666666666


In [23]:
from lightgbm import LGBMClassifier

# LightGBM Model
lgb_model = LGBMClassifier()
lgb_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_lgb = lgb_model.predict(X_test_scaled)

accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
precision_lgb = precision_score(y_test, y_pred_lgb, zero_division=1)
recall_lgb = recall_score(y_test, y_pred_lgb)
f1_lgb = f1_score(y_test, y_pred_lgb)

# Output the results for LightGBM
print(f'LightGBM Accuracy: {accuracy_lgb}')
print(f'LightGBM Precision: {precision_lgb}')
print(f'LightGBM Recall: {recall_lgb}')
print(f'LightGBM F1-Score: {f1_lgb}')


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 4, number of negative: 4
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 8, number of used features: 0
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[

In [24]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Model
rf_model = RandomForestClassifier(class_weight='balanced')
rf_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_rf = rf_model.predict(X_test_scaled)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, zero_division=1)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Output the results for Random Forest
print(f'Random Forest Accuracy: {accuracy_rf}')
print(f'Random Forest Precision: {precision_rf}')
print(f'Random Forest Recall: {recall_rf}')
print(f'Random Forest F1-Score: {f1_rf}')


Random Forest Accuracy: 0.6666666666666666
Random Forest Precision: 1.0
Random Forest Recall: 0.5
Random Forest F1-Score: 0.6666666666666666


In [25]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting Model
gbm_model = GradientBoostingClassifier()
gbm_model.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred_gbm = gbm_model.predict(X_test_scaled)

accuracy_gbm = accuracy_score(y_test, y_pred_gbm)
precision_gbm = precision_score(y_test, y_pred_gbm, zero_division=1)
recall_gbm = recall_score(y_test, y_pred_gbm)
f1_gbm = f1_score(y_test, y_pred_gbm)

# Output the results for GBM
print(f'GBM Accuracy: {accuracy_gbm}')
print(f'GBM Precision: {precision_gbm}')
print(f'GBM Recall: {recall_gbm}')
print(f'GBM F1-Score: {f1_gbm}')


GBM Accuracy: 0.6666666666666666
GBM Precision: 0.6666666666666666
GBM Recall: 1.0
GBM F1-Score: 0.8


In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold, train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd

# Load the dataset
df = pd.read_csv('video_features_with_labels.csv')

# Prepare features (X) and target (y)
X = df.drop('fraud', axis=1)
y = df['fraud']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SMOTE with k_neighbors=1 to handle the small dataset
smote = SMOTE(k_neighbors=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Define the neural network model
def create_nn():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train_smote.shape[1],)))
    model.add(Dropout(0.4))  # Dropout to prevent overfitting, try adjusting to 0.3 or 0.5
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Perform manual cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = []

for train_index, val_index in kf.split(X_train_smote):
    X_train_cv, X_val_cv = X_train_smote[train_index], X_train_smote[val_index]
    y_train_cv, y_val_cv = y_train_smote[train_index], y_train_smote[val_index]

    # Create and train the model
    model = create_nn()
    model.fit(X_train_cv, y_train_cv, epochs=10, batch_size=10, verbose=1)  # Increased epochs to 10

    # Evaluate the model
    val_predictions = model.predict(X_val_cv)
    val_predictions = (val_predictions > 0.5).astype(int)  # Convert to binary predictions
    accuracy = accuracy_score(y_val_cv, val_predictions)
    accuracy_scores.append(accuracy)

# Output the cross-validation accuracy scores
print(f'Cross-Validation Accuracy Scores: {accuracy_scores}')
print(f'Average Cross-Validation Accuracy: {np.mean(accuracy_scores)}')

# Train the model on the full training set with class_weight to balance classes
model = create_nn()
model.fit(X_train_smote, y_train_smote, epochs=10, batch_size=10, verbose=1, class_weight={0: 1, 1: 3})

# Predict using the test set
y_pred_nn = model.predict(X_test_scaled)
y_pred_nn = (y_pred_nn > 0.5).astype(int)  # Convert probabilities to binary class (0 or 1)

# Evaluate the model
accuracy_nn = accuracy_score(y_test, y_pred_nn)
precision_nn = precision_score(y_test, y_pred_nn, zero_division=1)
recall_nn = recall_score(y_test, y_pred_nn)
f1_nn = f1_score(y_test, y_pred_nn)

# Output the results
print(f'Neural Networks Accuracy: {accuracy_nn}')
print(f'Neural Networks Precision: {precision_nn}')
print(f'Neural Networks Recall: {recall_nn}')
print(f'Neural Networks F1-Score: {f1_nn}')

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_nn)
print("Confusion Matrix:")
print(cm)


C:\Users\sivad\Downloads\jupyter\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 945ms/step - accuracy: 0.5000 - loss: 0.7671
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.0118
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 2.2280e-05
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 9.3451e-07
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.0000 - loss: 1.7262e-05
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 1.8933e-04
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 5.1251e-11
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 6.5401e-07
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - loss: 1.0632e-09
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 2.3621e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Epoch 1/10


C:\Users\sivad\Downloads\jupyter\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6667 - loss: 0.8472
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8333 - loss: 0.2755
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.0792
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 2.6368e-04
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 3.1505e-04
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 6.7281e-06
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - loss: 0.0547
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 9.2726e-07
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 2.7564e-07
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 1.1566e-08
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Epoch 1/10


C:\Users\sivad\Downloads\jupyter\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4286 - loss: 0.7483
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0441
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0047
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8571 - loss: 0.1405
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 0.0110
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0217
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0012
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - loss: 0.0784
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 1.8427e-05
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 3.9138e-07
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Epoch 1/10


C:\Users\sivad\Downloads\jupyter\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5714 - loss: 0.8468
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5714 - loss: 2.8905
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8571 - loss: 0.4365
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8571 - loss: 0.2971
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8571 - loss: 0.1179
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 7.5012e-04
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8571 - loss: 0.1882
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0021
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 4.0384e-09
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 6.1535e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Cross-Validation Accuracy Scores: [0.5, 0.5, 0.5, 1.0, 1.0]
Average Cross-Validation Accuracy: 0.7
Epoch 1/10


C:\Users\sivad\Downloads\jupyter\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 2.1776
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8750 - loss: 0.8496
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 5.1443e-04
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8750 - loss: 0.9907
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7500 - loss: 3.0972
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0022
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0501
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.2098
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 4.2299e-10
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 1.1304e-07
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Neural Networks Accuracy: 1.0
Neural Networks Precision: 1.0
Neural Networks Recall: 1.0
Neural Networks F1-Score: 1.0
Confusion Mat